In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

In [2]:
startpath ="_Star_Ratings_and_Display_Measures/"
fallpath ="_Star_Ratings_Fall_Release/"
springpath ="_Star_Ratings_Spring_Release/"
cpath="_Part_C"
dpath="_Part_D"
finalpath ="_Report_Card_Master_Table.xlsx"
year = "2014"



In [3]:
firstfive = ["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization"]

In [19]:
def get_measure_stars(path):
    dfms = pd.read_excel(path,"Measure_Stars",header=2)
    dfms = dfms.iloc[1:]
    for x in range(5,len(dfms.keys())):
        dfms =dfms.rename(columns= {dfms.keys()[x] : dfms.keys()[x].split(': ')[1]})
    for x in range(5):
        dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
    return dfms

In [5]:
def get_domain_stars(path):
    dfds = pd.read_excel(path,"Domain_Stars",header=1)
    return dfds

In [6]:
def get_summary_rating(path):
    dfsr = pd.read_excel(path,"Summary_Rating",header=1)
    dfsr = dfsr.drop(columns="Sanction Deduction",errors='ignore')
    return dfsr

In [7]:
def get_early(y):
    fullpath = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
    dfmsf = get_measure_stars(fullpath)
    dfdsf = get_domain_stars(fullpath)
    dfsrf = get_summary_rating(fullpath)
    dff = pd.merge(dfmsf,dfdsf,on=firstfive,how='left')
    dff = pd.merge(dff,dfsrf,on=firstfive,how='left')
    dff["Year"] = f"{y} Fall"
    dff = dff.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    fullpath = f"./Data/{y}{startpath}{y}{springpath}{y}{finalpath}"
    dfmss = get_measure_stars(fullpath)
    dfdss = get_domain_stars(fullpath)
    dfsrs = get_summary_rating(fullpath)
    dfs = pd.merge(dfmss,dfdss,on=firstfive,how='left')
    dfs = pd.merge(dfs,dfsrs,on=firstfive,how='left')
    dfs["Year"] = f"{y} Spring"
    dfs = dfs.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df = pd.concat([dff,dfs])
    df = df.drop(columns = "2017 Disaster %",errors='ignore')
    return df

In [8]:
def get_late(y):
    fullpath = f"./Data/{y}{startpath}{y}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    #drop disaster columns
    dfsrc = dfsrc.drop(columns=dfsrc.columns[[6,7]],axis = 1)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    df = df.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df["Year"] = f"{y}"
    return    df

In [24]:
def standardize_data():
    year = 2014
    years = [x for x in range(2015,2026)]        
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{cpath}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{dpath}{finalpath}"
    dfmsd = get_measure_stars(fullpath)
    dfdsd = get_domain_stars(fullpath)
    dfsrd = get_summary_rating(fullpath)
    df = pd.merge(df,dfmsd,on=firstfive,how='left')
    df = pd.merge(df,dfdsd,on=firstfive,how='left')
    df = pd.merge(df,dfsrd,on=firstfive,how='left')
    df = df.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall",f"{year} Part D Summary Rating":"Part D Summary"})
    fullpath = f"./Data/{year}{startpath}{year}{springpath}{year}{cpath}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    df1 = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df1 = pd.merge(df1,dfsrc,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{springpath}{year}{dpath}{finalpath}"
    dfmsd = get_measure_stars(fullpath)
    dfdsd = get_domain_stars(fullpath)
    dfsrd = get_summary_rating(fullpath)
    df1 = pd.merge(df1,dfmsd,on=firstfive,how='left')
    df1 = pd.merge(df1,dfdsd,on=firstfive,how='left')
    df1 = pd.merge(df1,dfsrd,on=firstfive,how='left')    
    df1 = df1.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall","{year} Part D Summary Rating":"Part D Summary"})    
    df["Year"] = "2014 Fall"
    df1["Year"] = "2014 Spring"
    finaldf = pd.concat([df,df1])
    spark = SparkSession.builder.appName("pandas to spark").getOrCreate()
    sparkdf = spark.createDataFrame(finaldf)
    for year in years:
        if (year <2020):
            df = get_early(year)
        else:
            df = get_late(year)
        placeholderdf = spark.createDataFrame(df)
        newsparkdf = sparkdf.unionByName(placeholderdf,allowMissingColumns=True)
        sparkdf = newsparkdf
            #finaldf = pd.concat(finaldf,df)
    return sparkdf

In [25]:
combodf = standardize_data()
combodf.show()

AnalysisException: [COLUMN_ALREADY_EXISTS] The column `members choosing to leave the plan` already exists. Consider to choose another name or rename the existing column.

In [ ]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

data1=[
(1 , '2016-08-29', 1 , 2, 3),
(2 , '2016-08-29', 1 , 2, 3),
(3 , '2016-08-29', 1 , 2, 3)]
df1 = spark.createDataFrame(data1, ['code' , 'date' , 'A' , 'B', 'C'])
data2=[
(5 , '2016-08-29', 1, 2, 3, 4),
(6 , '2016-08-29', 1, 2, 3, 4),
(7 , '2016-08-29', 1, 2, 3, 4)]
df2 = spark.createDataFrame(data2, ['code' , 'date' , 'B', 'C', 'D', 'E'])

diff1 = [c for c in df2.columns if c not in df1.columns]
diff2 = [c for c in df1.columns if c not in df2.columns]
df = df1.select('*', *[F.lit(None).alias(c) for c in diff1]) \
    .unionByName(df2.select('*', *[F.lit(None).alias(c) for c in diff2]))
df.show()

+----+----------+----+---+---+----+----+
|code|      date|   A|  B|  C|   D|   E|
+----+----------+----+---+---+----+----+
|   1|2016-08-29|   1|  2|  3|NULL|NULL|
|   2|2016-08-29|   1|  2|  3|NULL|NULL|
|   3|2016-08-29|   1|  2|  3|NULL|NULL|
|   5|2016-08-29|NULL|  1|  2|   3|   4|
|   6|2016-08-29|NULL|  1|  2|   3|   4|
|   7|2016-08-29|NULL|  1|  2|   3|   4|
+----+----------+----+---+---+----+----+



25/02/24 01:08:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 948160 ms exceeds timeout 120000 ms
25/02/24 01:08:13 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/24 01:08:18 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [61]:
spark1 = SparkSession.builder.appName("pandas to spark").getOrCreate()

testdf = get_late(2020)
testdf2 = get_early(2016)
sparktestdf =spark1.createDataFrame(testdf)
sparktestdf2 =spark1.createDataFrame(testdf2)
sparktestdf =sparktestdf.union(sparktestdf2)
sparktestdf.show()

+---------------+--------------------+--------------------+---------------------------+--------------------+----------------------------+--------------------------------+-----------------------+---------------------------------------------+-------------------------------------------+---------------------------------+-------------------------+---------------------------------------------+----------------------------------------------+---------------------------------------------------------+--------------------------------------------+--------------------------------------------------------+-----------------------------+----------------------------------------------+-------------------------------------------+------------------------------------+---------------------------------+------------------------------+---------------------------------------------+--------------------------------+------------------------------------------------------------+------------------------+---------------

In [62]:
sparktestdf.count()

2027

In [60]:
testdf2.describe()

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C01: Breast Cancer Screening,C02: Colorectal Cancer Screening,C03: Annual Flu Vaccine,C04: Improving or Maintaining Physical Health,C05: Improving or Maintaining Mental Health,...,HD5: Health Plan Customer Service,DD1: Drug Plan Customer Service,DD2: Member Complaints and Changes in the Drug Plan’s Performance,DD3: Member Experience with the Drug Plan,DD4: Drug Safety and Accuracy of Drug Pricing,SNP,Part C Summary,Part D Summary,Overall,Year
count,1284,1284,1284,1284,1284,1284,1284,1284,1284,1284,...,1284,1284,1284,1284,1284,1284,1284,1284,1284,1284
unique,642,8,469,413,222,10,10,9,9,9,...,8,8,7,8,7,2,11,10,9,2
top,E0654,Local CCP,UNITEDHEALTHCARE INSURANCE COMPANY,UnitedHealthcare,"UnitedHealth Group, Inc.",3,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,Not enough data available,5,4,Plan too new to be measured,4,No,Plan too new to be measured,4,Plan too new to be measured,2016 Fall
freq,2,938,28,78,114,292,244,244,412,412,...,370,446,440,252,622,820,244,284,238,642
